In [1]:
import pandas as pd
import cudf as cd
import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

from cuml.preprocessing import LabelEncoder

from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from cuml.metrics.accuracy import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB as skNB
from sklearn.neighbors import KNeighborsClassifier as skKNN
from cuml.naive_bayes import GaussianNB as cuNB
from cuml.neighbors import KNeighborsClassifier as cuKNN
from cuml.linear_model import LogisticRegression as cuLR

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
df = cd.read_parquet("data/BaIoT/combined_sample.parquet")
df

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,traffic_type
0,141.567688,353.580963,5.883921e+04,237.540222,353.870087,5.881219e+04,680.254883,370.407135,56988.163829,6468.260254,...,0.0,0.0,1.000000,554.0,0.000000e+00,554.000000,0.000000e+00,0.0,0.0,mirai_udp
1,1.000000,60.000000,0.000000e+00,1.000000,60.000000,0.000000e+00,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,bl_tcp
2,154.181931,74.285988,6.597754e+01,259.213043,74.462685,1.091654e+02,791.391846,74.428703,100.003105,7849.097168,...,0.0,0.0,1.687602,74.0,9.536743e-07,95.268044,9.094947e-13,0.0,0.0,bl_combo
3,45.116440,74.257240,6.107529e+00,76.664810,74.225479,5.362086e+00,197.098373,74.230797,11.322430,1192.187134,...,0.0,0.0,2.898656,74.0,1.348699e-06,74.000000,1.818989e-12,0.0,0.0,bl_scan
4,83.959778,60.000000,2.728484e-12,100.172050,60.000000,1.464468e-08,213.646896,60.000095,0.000580,1935.140625,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_scan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412516,113.422058,419.350372,4.838640e+04,185.200226,447.269287,4.133353e+04,583.623230,447.477325,41274.304743,6226.459473,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_udp
1412517,150.834183,74.559334,1.288838e+02,258.264587,74.685402,1.615611e+02,793.014709,74.487106,113.598282,7849.941895,...,0.0,0.0,1.687182,74.0,0.000000e+00,95.268044,0.000000e+00,0.0,0.0,bl_combo
1412518,166.402374,335.931335,6.017198e+04,228.481537,365.908722,5.753876e+04,594.888489,405.546631,51297.564506,6469.588379,...,0.0,0.0,1.000000,60.0,0.000000e+00,60.000000,0.000000e+00,0.0,0.0,mirai_udp
1412519,165.369736,74.090378,2.188083e+00,273.042328,74.071251,1.816287e+00,819.331909,74.044846,1.356601,7850.386230,...,0.0,0.0,1.687282,74.0,0.000000e+00,95.268044,0.000000e+00,0.0,0.0,bl_combo


In [4]:
X = df.drop('traffic_type', axis=1)
y = LabelEncoder().fit_transform(df['traffic_type'])
del df

In [7]:
def gpu_cross_validate(clf, X, y, cv=5):
    splitter = StratifiedKFold(n_splits=cv)
    splits = splitter.split(np.zeros(len(X)), y.to_numpy())
    for train, test in splits:
        model = clone(clf)
        print("training started")
        model.fit(X.iloc[train], y.iloc[train])

        print("predicting started")
        y_pred = model.predict(X.iloc[test])

        print(f"accuracy: {accuracy_score(y.iloc[test], y_pred)}")
        del model, 


In [8]:
gpu_cross_validate(cuKNN(), X, y)

[I] [22:48:27.703632] Unused keyword parameter: n_jobs during cuML estimator initialization
training started
predicting started
accuracy: 0.5552008152008057
[I] [22:48:57.672245] Unused keyword parameter: n_jobs during cuML estimator initialization
training started
predicting started
accuracy: 0.6141718626022339
[I] [22:49:27.399970] Unused keyword parameter: n_jobs during cuML estimator initialization
training started
predicting started
accuracy: 0.613775372505188
[I] [22:49:58.157110] Unused keyword parameter: n_jobs during cuML estimator initialization
training started


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/arisingh/mambaforge/envs/ccd/include/rmm/mr/device/cuda_memory_resource.hpp

In [ ]:
gpu_cross_validate(cuLR(), X, y)